In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import deque
import gc
import csv
import requests
import math
import base64
from datetime import datetime
from twilio.rest import Client
import sendgrid
from sendgrid.helpers.mail import Mail, Attachment, FileContent, FileName, FileType, Disposition

# --- Twilio Credentials ---
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env

TWILIO_SID = os.getenv("TWILIO_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")
RECIPIENT_PHONE_NUMBER = os.getenv("RECIPIENT_PHONE_NUMBER")



# --- SendGrid API ---
import os
sendgrid_key = os.getenv("SENDGRID_API_KEY")


# --- Load YOLOv8 Models ---
video_path = os.getenv("VIDEO_PATH")
model_nano = YOLO(os.getenv("MODEL_NANO_PATH"))
model_medium = YOLO(os.getenv("MODEL_MEDIUM_PATH"))

# --- Parameters ---
CONF_THRESHOLD = 0.3
CRASH_CONF_THRESHOLD = 0.5
TEMPORAL_WINDOW = 5
FRAME_WIDTH, FRAME_HEIGHT = 640, 480
FRAME_SKIP = 2
class_id_crash = 0
class_id_vehicle = 1
temporal_buffer = deque(maxlen=TEMPORAL_WINDOW)

last_boxes, last_scores, last_classes = [], [], []
vehicle_data = {}
vehicle_tracks = {}
alert_sent = False

# --- Speed Estimation Parameters ---
# Assuming the standard lane width is 3.7 meters
LANE_WIDTH_METERS = 3.7
# Estimated lane width in pixels 
LANE_WIDTH_PIXELS = 120
# Conversion factor (meters per pixel)
METERS_PER_PIXEL = LANE_WIDTH_METERS / LANE_WIDTH_PIXELS
# Speed smoothing buffer size
SPEED_BUFFER_SIZE = 5

# --- Video Paths ---
video_path = os.path.join("video_final", "Crash_5.mp4")
cap = cv2.VideoCapture(video_path)
FPS = cap.get(cv2.CAP_PROP_FPS)

# --- Output Video Writer ---
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_crash_detection.mp4', fourcc, FPS, (FRAME_WIDTH, FRAME_HEIGHT))

# --- Functions ---
def clear_gpu():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

def detect_objects(model, frame):
    result = model(frame, verbose=False)[0]
    boxes, scores, classes = [], [], []

    for box, conf, cls in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
        if conf < CONF_THRESHOLD:
            continue
        x1, y1, x2, y2 = map(int, box.tolist())
        width, height = x2 - x1, y2 - y1

        if int(cls) == class_id_crash and conf >= CRASH_CONF_THRESHOLD:
            if width >= FRAME_WIDTH * 0.8 or height >= FRAME_HEIGHT * 0.8:
                continue

        boxes.append([x1, y1, x2, y2])
        scores.append(float(conf))
        classes.append(int(cls))

    return boxes, scores, classes

def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1p, y1p, x2p, y2p = box2
    inter_x1, inter_y1 = max(x1, x1p), max(y1, y1p)
    inter_x2, inter_y2 = min(x2, x2p), min(y2, y2p)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x2p - x1p) * (y2p - y1p)
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0

def match_crashes(boxes_nano, boxes_medium, iou_threshold=0.5):
    confirmed = []
    for box_n in boxes_nano:
        for box_m in boxes_medium:
            if iou(box_n, box_m) > iou_threshold:
                confirmed.append(box_n)
                break
    return confirmed

def send_twilio_alert(detected_vehicles=None, location="Ahmedabad, India"):
    try:
        # Get current time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Prepare detailed message
        message_body = f"⚠️ ALERT! Accident Detected!\n"
        message_body += f"Time: {current_time}\n"
        message_body += f"Location: {location}\n\n"
        
        # Add vehicle information if available
        if detected_vehicles and len(detected_vehicles) > 0:
            message_body += "Vehicle information:\n"
            for track_id, data in detected_vehicles.items():
                if "max_speed" in data and "crash_probability" in data:
                    message_body += f"Vehicle #{track_id}: Speed {data.get('max_speed', 0)}km/h, "
                    message_body += f"Risk level: {data.get('crash_probability', 0)}%\n"
                    
                    # Add driving pattern info if we have it
                    if track_id in vehicle_tracks and "speeds" in vehicle_tracks[track_id] and len(vehicle_tracks[track_id]["speeds"]) >= 3:
                        speeds = list(vehicle_tracks[track_id]["speeds"])
                        speed_changes = [abs(speeds[i] - speeds[i-1]) for i in range(1, len(speeds))]
                        avg_change = sum(speed_changes) / len(speed_changes)
                        
                        if avg_change > 15:
                            message_body += "Driving pattern: Erratic acceleration/braking\n"
                        elif avg_change > 8:
                            message_body += "Driving pattern: Rapid speed changes\n"
                        else:
                            message_body += "Driving pattern: Normal\n"
        
        # Send SMS
        client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
        message = client.messages.create(
            body=message_body,
            from_=TWILIO_PHONE_NUMBER,
            to=RECIPIENT_PHONE_NUMBER
        )
        print("📱 Twilio alert sent with detailed information.")
    except Exception as e:
        print("⚠️ Failed to send Twilio alert:", e)

def send_email_alert(detected_vehicles=None, location="Ahmedabad, India", current_frame=None):
    try:
        # Get current time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Prepare detailed HTML content
        html_content = f"""
        <h2>⚠️ ACCIDENT ALERT!</h2>
        <p><strong>Time:</strong> {current_time}</p>
        <p><strong>Location:</strong> {location}</p>
        """
        
        # Add vehicle information if available
        if detected_vehicles and len(detected_vehicles) > 0:
            html_content += "<h3>Vehicle Information:</h3><table border='1'>"
            html_content += "<tr><th>Vehicle ID</th><th>Speed</th><th>Risk Level</th><th>Driving Pattern</th></tr>"
            
            for track_id, data in detected_vehicles.items():
                if "max_speed" in data and "crash_probability" in data:
                    speed = data.get('max_speed', 0)
                    risk = data.get('crash_probability', 0)
                    
                    # Determine driving pattern
                    pattern = "Normal"
                    if track_id in vehicle_tracks and "speeds" in vehicle_tracks[track_id] and len(vehicle_tracks[track_id]["speeds"]) >= 3:
                        speeds = list(vehicle_tracks[track_id]["speeds"])
                        speed_changes = [abs(speeds[i] - speeds[i-1]) for i in range(1, len(speeds))]
                        avg_change = sum(speed_changes) / len(speed_changes)
                        
                        if avg_change > 15:
                            pattern = "Erratic acceleration/braking"
                        elif avg_change > 8:
                            pattern = "Rapid speed changes"
                    
                    # Add row to table
                    html_content += f"<tr><td>{track_id}</td><td>{speed} km/h</td>"
                    html_content += f"<td>{risk}%</td><td>{pattern}</td></tr>"
            
            html_content += "</table>"
            
            # Add image if available
            html_content += "<p>Please check the attached video frame for more details.</p>"
        
        # Create email
        sg = sendgrid.SendGridAPIClient(api_key=SENDGRID_API_KEY)
        
        mail = Mail(
            from_email="khuship.study@gmail.com",
            to_emails="kmp.272003@gmail.com",
            subject="🚨 Accident Alert - Immediate Attention Required",
            html_content=html_content
        )
        
        # Optionally attach current video frame
        try:
            if current_frame is not None:
                frame_path = "accident_frame.jpg"
                cv2.imwrite(frame_path, current_frame)
                
                # Attach to email
                with open(frame_path, 'rb') as f:
                    data = f.read()
                    f.close()
                
                encoded = base64.b64encode(data).decode()
                attachment = Attachment()
                attachment.file_content = FileContent(encoded)
                attachment.file_name = FileName('accident_scene.jpg')
                attachment.file_type = FileType('image/jpeg')
                attachment.disposition = Disposition('attachment')
                mail.attachment = attachment
        except Exception as img_error:
            print(f"Could not attach image: {img_error}")
        
        # Send email
        sg.send(mail)
        print("📧 Email alert sent with detailed information and image.")
    except Exception as e:
        print("⚠️ Failed to send email alert:", e)

def estimate_speed(track_id, center_x, center_y, box_width, box_height, timestamp):
    """
    Calculate speed more accurately using physical dimensions and time
    
    Args:
        track_id: Vehicle identifier
        center_x, center_y: Current position
        box_width, box_height: Bounding box dimensions
        timestamp: Current frame timestamp
    
    Returns:
        Estimated speed in km/h
    """
    if track_id not in vehicle_tracks:
        # Initialize new vehicle track with position, time, and empty speed buffer
        vehicle_tracks[track_id] = {
            "positions": [(center_x, center_y)],
            "timestamps": [timestamp],
            "speeds": deque(maxlen=SPEED_BUFFER_SIZE),
            "last_speed": 0
        }
        return 0
    
    # Get track data
    track_data = vehicle_tracks[track_id]
    prev_positions = track_data["positions"]
    prev_timestamps = track_data["timestamps"]
    
    # Need at least two points to calculate speed
    if len(prev_positions) < 1:
        track_data["positions"].append((center_x, center_y))
        track_data["timestamps"].append(timestamp)
        return 0
    
    # Calculate time difference in seconds
    prev_time = prev_timestamps[-1]
    time_diff = (timestamp - prev_time)
    
    # Ensure we don't divide by zero
    if time_diff <= 0:
        return track_data["last_speed"]
    
    # Get previous position
    prev_x, prev_y = prev_positions[-1]
    
    # Calculate pixel distance
    distance_px = math.sqrt((center_x - prev_x)**2 + (center_y - prev_y)**2)
    
    # Convert to meters using calibration factor
    distance_m = distance_px * METERS_PER_PIXEL
    
    # Apply perspective correction - objects farther up in frame appear smaller
    # Simple correction based on y-position (higher y = closer to horizon = farther away)
    perspective_factor = 1.0 + (1.0 - center_y / FRAME_HEIGHT) * 0.5
    distance_m *= perspective_factor
    
    # Calculate speed in m/s
    speed_ms = distance_m / time_diff
    
    # Convert to km/h
    speed_kmh = speed_ms * 3.6
    
    # Apply size-based correction - smaller vehicles (farther away) need speed adjustment
    size_factor = math.sqrt((box_width * box_height) / (FRAME_WIDTH * FRAME_HEIGHT))
    size_correction = 1.0 + (0.2 / max(size_factor, 0.01))
    speed_kmh *= min(size_correction, 2.0)  # Limit correction factor
    
    # Apply physics-based constraints
    # 1. Limit acceleration/deceleration
    if track_data["last_speed"] > 0:
        max_acceleration = 8  # km/h per second
        max_change = max_acceleration * time_diff
        speed_kmh = max(0, min(speed_kmh, track_data["last_speed"] + max_change))
        speed_kmh = max(0, max(speed_kmh, track_data["last_speed"] - max_change))
    
    # 2. Speed limits based on context
    min_speed = 0
    max_speed = 90  # km/h - highway
    speed_kmh = max(min_speed, min(speed_kmh, max_speed))
    
    # Add to speed buffer for smoothing
    track_data["speeds"].append(speed_kmh)
    
    # Calculate smoothed speed using moving average
    if len(track_data["speeds"]) > 0:
        smoothed_speed = sum(track_data["speeds"]) / len(track_data["speeds"])
    else:
        smoothed_speed = speed_kmh
    
    # Update track data
    track_data["positions"].append((center_x, center_y))
    track_data["timestamps"].append(timestamp)
    track_data["last_speed"] = int(smoothed_speed)
    
    # Keep only recent positions (last 10)
    if len(track_data["positions"]) > 10:
        track_data["positions"] = track_data["positions"][-10:]
        track_data["timestamps"] = track_data["timestamps"][-10:]
    
    return int(smoothed_speed)

def calculate_crash_probability(speed, track_id, current_vehicles):
    """
    Calculate crash probability based on multiple risk factors:
    - Speed
    - Acceleration/deceleration
    - Proximity to other vehicles
    - Lane position/behavior
    - Historical driving pattern
    """
    # Base probability from speed (similar to original logic but refined)
    if speed < 30:
        base_prob = 5  # Even slow vehicles have some crash risk
    elif 30 <= speed < 60:
        base_prob = 15
    elif 60 <= speed < 90:
        base_prob = 30
    elif 90 <= speed < 120:
        base_prob = 60
    else:
        base_prob = 80
    
    # Get track data for this vehicle
    track_data = vehicle_tracks.get(track_id, None)
    if track_data is None or len(track_data.get("positions", [])) < 3:
        return base_prob  # Not enough data for advanced calculation
    
    total_prob = base_prob
    factors_applied = 1
    
    # --- Factor 1: Sudden acceleration/deceleration ---
    if "speeds" in track_data and len(track_data["speeds"]) >= 3:
        speeds = list(track_data["speeds"])
        # Calculate speed changes
        speed_changes = [abs(speeds[i] - speeds[i-1]) for i in range(1, len(speeds))]
        avg_change = sum(speed_changes) / len(speed_changes)
        
        # Penalize rapid speed changes
        if avg_change > 15:  # km/h per frame is very abrupt
            total_prob += 25
            factors_applied += 1
        elif avg_change > 8:
            total_prob += 15
            factors_applied += 1
    
    # --- Factor 2: Proximity to other vehicles ---
    current_pos = track_data["positions"][-1]
    closest_distance = float('inf')
    
    for other_id in current_vehicles:
        if other_id == track_id or other_id not in vehicle_tracks:
            continue
            
        other_data = vehicle_tracks[other_id]
        if not other_data.get("positions"):
            continue
            
        other_pos = other_data["positions"][-1]
        distance = ((current_pos[0] - other_pos[0])**2 + (current_pos[1] - other_pos[1])**2)**0.5
        
        # Convert pixel distance to approximate meters
        distance_meters = distance * METERS_PER_PIXEL
        
        if distance_meters < closest_distance:
            closest_distance = distance_meters
    
    # Add proximity risk
    if closest_distance != float('inf'):
        if closest_distance < 1.5:  # Very dangerous proximity
            total_prob += 40
            factors_applied += 1
        elif closest_distance < 3:  # Unsafe following distance
            total_prob += 20
            factors_applied += 1
        elif closest_distance < 5:  # Close but not dangerous
            total_prob += 10
            factors_applied += 1
    
    # --- Factor 3: Erratic movement patterns ---
    if len(track_data["positions"]) >= 5:
        # Calculate path smoothness
        points = track_data["positions"][-5:]
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        
        # Calculate variance of the changes in direction
        dx = [x_coords[i] - x_coords[i-1] for i in range(1, len(x_coords))]
        dy = [y_coords[i] - y_coords[i-1] for i in range(1, len(y_coords))]
        
        # Calculate angles
        angles = []
        for i in range(len(dx)-1):
            if dx[i] == 0:  # Avoid division by zero
                dx[i] = 0.001
            if dx[i+1] == 0:
                dx[i+1] = 0.001
                
            angle1 = math.atan2(dy[i], dx[i])
            angle2 = math.atan2(dy[i+1], dx[i+1])
            angle_diff = abs(angle1 - angle2)
            angles.append(angle_diff)
        
        if angles:
            avg_angle_change = sum(angles) / len(angles)
            
            # Convert to degrees for easier interpretation
            avg_angle_change_deg = math.degrees(avg_angle_change)
            
            # Penalize erratic movements
            if avg_angle_change_deg > 30:  # Very erratic
                total_prob += 30
                factors_applied += 1
            elif avg_angle_change_deg > 15:  # Somewhat erratic
                total_prob += 15
                factors_applied += 1
    
    # --- Factor 4: Edge case - proximity to frame edge ---
    x, y = current_pos
    edge_margin = 50  # pixels
    
    if x < edge_margin or x > FRAME_WIDTH - edge_margin or \
       y < edge_margin or y > FRAME_HEIGHT - edge_margin:
        # Vehicle near the edge - could be entering/exiting frame or at road edge
        total_prob += 10
        factors_applied += 1
    
    # Calculate weighted average
    final_prob = min(total_prob / factors_applied, 100)
    
    # Round to nearest integer
    return int(round(final_prob))


def generate_report(filename, location="Ahmedabad, India"):
    """Generate CSV report with vehicle data and accident information"""
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Location", "Vehicle ID", "Speed (km/h)", "Crash Probability (%)", "Risk Factors"])
        
        for track_id, data in vehicle_data.items():
            # Generate risk factor string based on stored data
            risk_factors = []
            
            # Add speed as risk factor if high
            if data.get("max_speed", 0) > 80:
                risk_factors.append("High Speed")
            
            # Check if we have acceleration data
            if track_id in vehicle_tracks and "speeds" in vehicle_tracks[track_id] and len(vehicle_tracks[track_id]["speeds"]) >= 3:
                speeds = list(vehicle_tracks[track_id]["speeds"])
                speed_changes = [abs(speeds[i] - speeds[i-1]) for i in range(1, len(speeds))]
                avg_change = sum(speed_changes) / len(speed_changes)
                
                if avg_change > 15:
                    risk_factors.append("Erratic Acceleration/Braking")
                elif avg_change > 8:
                    risk_factors.append("Rapid Speed Changes")
            
            # If we have position data, check for erratic movement
            if track_id in vehicle_tracks and len(vehicle_tracks[track_id].get("positions", [])) >= 5:
                points = vehicle_tracks[track_id]["positions"][-5:]
                x_coords = [p[0] for p in points]
                y_coords = [p[1] for p in points]
                
                dx = [x_coords[i] - x_coords[i-1] for i in range(1, len(x_coords))]
                dy = [y_coords[i] - y_coords[i-1] for i in range(1, len(y_coords))]
                
                angles = []
                for i in range(len(dx)-1):
                    if dx[i] == 0:
                        dx[i] = 0.001
                    if dx[i+1] == 0:
                        dx[i+1] = 0.001
                        
                    angle1 = math.atan2(dy[i], dx[i])
                    angle2 = math.atan2(dy[i+1], dx[i+1])
                    angle_diff = abs(angle1 - angle2)
                    angles.append(angle_diff)
                
                if angles:
                    avg_angle_change = sum(angles) / len(angles)
                    avg_angle_change_deg = math.degrees(avg_angle_change)
                    
                    if avg_angle_change_deg > 30:
                        risk_factors.append("Very Erratic Movement")
                    elif avg_angle_change_deg > 15:
                        risk_factors.append("Erratic Movement")
            
            # Add any proximity issues as risk factor
            if data.get("crash_probability", 0) > 50 and "max_speed" in data and data["max_speed"] < 70:
                risk_factors.append("Close Proximity to Vehicles")
            
            # If no specific factors found
            if not risk_factors:
                risk_factors.append("Normal Driving")
            
            writer.writerow([
                current_time, 
                location, 
                track_id, 
                data.get("max_speed", 0), 
                data.get("crash_probability", 0),
                ", ".join(risk_factors)
            ])

        if alert_sent:
            writer.writerow([])
            writer.writerow(["ALERT: An accident was detected in the video!"])
            writer.writerow([f"Timestamp: {current_time}"])
            writer.writerow([f"Location: {location}"])
    
    print(f"\n📜 Report saved to: {filename}")

# --- Main Loop ---
frame_id = 0
next_track_id = 1  # For assigning IDs to vehicles
start_time = datetime.now()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("🎥 End of video.")
        break

    frame_id += 1
    current_time = (datetime.now() - start_time).total_seconds()
    frame_resized = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))

    if frame_id % FRAME_SKIP == 0:
        boxes_nano, scores_nano, classes_nano = detect_objects(model_nano, frame_resized)
        boxes_med, scores_med, classes_med = detect_objects(model_medium, frame_resized)
        clear_gpu()

        crash_boxes_nano = [b for b, s, c in zip(boxes_nano, scores_nano, classes_nano) if c == class_id_crash and s >= CRASH_CONF_THRESHOLD]
        crash_boxes_med = [b for b, s, c in zip(boxes_med, scores_med, classes_med) if c == class_id_crash and s >= CRASH_CONF_THRESHOLD]

        confirmed_crashes = match_crashes(crash_boxes_nano, crash_boxes_med)
        accident_detected = len(confirmed_crashes) > 0
        temporal_buffer.append(accident_detected)

        # Process vehicle detections for speed estimation
        vehicle_boxes = [(b, s) for b, s, c in zip(boxes_nano, scores_nano, classes_nano) if c == class_id_vehicle]
        
        # Keep track of detected vehicles in this frame
        current_vehicles = set()
        
        for box, score in vehicle_boxes:
            x1, y1, x2, y2 = box
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            box_width = x2 - x1
            box_height = y2 - y1
            
            # Simple tracking - assign or match IDs based on position
            assigned = False
            min_distance = float('inf')
            best_track_id = None
            
            for tid in list(vehicle_tracks.keys()):
                # Only consider vehicles tracked recently
                if len(vehicle_tracks[tid]["positions"]) == 0:
                    continue
                    
                tx, ty = vehicle_tracks[tid]["positions"][-1]
                
                # If close to existing tracked vehicle, assume it's the same one
                distance = math.sqrt((center_x - tx) ** 2 + (center_y - ty) ** 2)
                
                # Find the closest vehicle within threshold
                if distance < 70 and distance < min_distance:  # Increased threshold for better tracking
                    min_distance = distance
                    best_track_id = tid
            
            if best_track_id is not None:
                track_id = best_track_id
                assigned = True
            
            if not assigned:
                track_id = next_track_id
                next_track_id += 1
            
            current_vehicles.add(track_id)
            
            # Calculate speed with improved method
            speed = estimate_speed(track_id, center_x, center_y, box_width, box_height, current_time)
            
            # Use new crash probability function - pass current_vehicles for proximity calculations
            crash_probability = calculate_crash_probability(speed, track_id, current_vehicles)
            
            # Update vehicle data for reporting
            if track_id not in vehicle_data:
                vehicle_data[track_id] = {"max_speed": speed, "crash_probability": crash_probability}
            else:
                vehicle_data[track_id]["max_speed"] = max(vehicle_data[track_id]["max_speed"], speed)
                vehicle_data[track_id]["crash_probability"] = max(vehicle_data[track_id]["crash_probability"], crash_probability)

        # Remove vehicles not seen for some time
        for tid in list(vehicle_tracks.keys()):
            if tid not in current_vehicles and len(vehicle_tracks[tid]["timestamps"]) > 0:
                last_seen = vehicle_tracks[tid]["timestamps"][-1]
                if current_time - last_seen > 3.0:  # Remove after 3 seconds of not being seen
                    vehicle_tracks.pop(tid, None)

        last_boxes, last_scores, last_classes = boxes_nano, scores_nano, classes_nano

    smoothed_crash = temporal_buffer.count(True) > TEMPORAL_WINDOW // 2

    if smoothed_crash and not alert_sent:
        send_twilio_alert()
        send_email_alert()
        alert_sent = True

    # Draw detections
    for i, (box, cls) in enumerate(zip(last_boxes, last_classes)):
        x1, y1, x2, y2 = box
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
        box_width = x2 - x1
        box_height = y2 - y1
        
        if cls == class_id_crash:
            color = (0, 0, 255)  # Red for crash
            label = "Crash"
            # Draw thick red box for crashes
            cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 3)
            
        else:
            color = (0, 255, 0)  # Green for vehicle
            
            # Find the track_id for this vehicle by position
            track_id = None
            min_distance = float('inf')
            
            for tid in vehicle_tracks:
                if len(vehicle_tracks[tid]["positions"]) == 0:
                    continue
                    
                tx, ty = vehicle_tracks[tid]["positions"][-1]
                distance = math.sqrt((center_x - tx) ** 2 + (center_y - ty) ** 2)
                
                if distance < 70 and distance < min_distance:
                    min_distance = distance
                    track_id = tid
            
            if track_id is not None and track_id in vehicle_tracks:
                speed = vehicle_tracks[track_id]["last_speed"]
                
                # Get crash probability for this vehicle
                if track_id in vehicle_data:
                    crash_prob = vehicle_data[track_id]["crash_probability"]
                else:
                    crash_prob = calculate_crash_probability(speed, track_id, current_vehicles)
                
                label = f"Vehicle {track_id}"
                
                # Color-code based on crash probability instead of just speed
                if crash_prob > 70:
                    color = (0, 0, 255)  # Red for high crash risk
                elif crash_prob > 40:
                    color = (0, 165, 255)  # Orange for medium crash risk
                elif crash_prob > 20:
                    color = (0, 255, 255)  # Yellow for low crash risk
                
                # Draw rectangle
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
                
                # # Draw speed gauge to the right of vehicle
                # if x2 + 60 < FRAME_WIDTH:
                #     draw_speed_gauge(frame_resized, speed, x2 + 30, y1 + 30)
                # else:
                #     # If not enough space on right, draw on left
                #     draw_speed_gauge(frame_resized, speed, x1 - 30, y1 + 30)
                
                # Display speed text and crash probability
                cv2.putText(frame_resized, f"{speed} km/h", (x1, y2 + 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                           
                cv2.putText(frame_resized, f"Risk: {crash_prob}%", (x1, y2 + 40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            else:
                label = "Vehicle"
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
        
        # Draw label
        cv2.putText(frame_resized, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    if smoothed_crash:
        # Draw prominent crash alert
        overlay = frame_resized.copy()
        cv2.rectangle(overlay, (0, 0), (FRAME_WIDTH, 50), (0, 0, 255), -1)
        cv2.addWeighted(overlay, 0.5, frame_resized, 0.5, 0, frame_resized)
        cv2.putText(frame_resized, "CRASH DETECTED", (10, 35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)

    # Add timestamp
    cv2.putText(frame_resized, f"Time: {current_time:.1f}s", (FRAME_WIDTH - 150, FRAME_HEIGHT - 10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    out.write(frame_resized)
    cv2.imshow("Crash Detection", frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 Interrupted by user.")
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

# Generate final report
report_path = r"C:\Users\Khushi Patel\OneDrive\Desktop\New_VSCODE\car\detection_report.csv"
generate_report(report_path)

print("\n✅ Detection Completed!")

🛑 Interrupted by user.

📜 Report saved to: C:\Users\Khushi Patel\OneDrive\Desktop\New_VSCODE\car\detection_report.csv

✅ Detection Completed!
